mcp/
 ├─ agents/
 │   ├─ forecast_agent.py
 │   ├─ reorder_agent.py
 │   ├─ inventory_agent.py
 │   ├─ routing_agent.py
 │   ├─ guardrails.py
 │   ├─ product_resolver.py
 │
 ├─ database/
 │   ├─ sqlite_handler.py
 │
 ├─ graph/
 │   ├─ workflow.py
 │
 ├─ llm/
 │   ├─ azure_llm.py
 │
 ├─ vectorstore/
 │   ├─ chroma_store.py
 │
 ├─ data/
 │   ├─ sales.db        # created by init_db.py
 │
 ├─ init_db.py
 ├─ run.py
 ├─ requirements.txt
 ├─ .env


langgraph
langchain
langchain-openai
python-dotenv
pandas
pydantic
numpy
chromadb
sentence-transformers


In [ ]:
# mcp/init_db.py
import sqlite3
from pathlib import Path

DB_PATH = Path("mcp/data/sales.db")
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS sales_history (
    date TEXT,
    sku TEXT,
    quantity INTEGER,
    price REAL,
    promo INTEGER,
    stockout INTEGER,
    region TEXT
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    sku TEXT PRIMARY KEY,
    name TEXT,
    stock INTEGER,
    threshold INTEGER
);
""")

cur.execute("DELETE FROM sales_history;")
cur.execute("DELETE FROM inventory;")

sales_data = [
    # date, sku, qty, price, promo, stockout, region
    ("2023-01-01", "SKU_SHAMPOO", 40, 19.99, 0, 0, "NORTH"),
    ("2023-02-01", "SKU_SHAMPOO", 45, 19.99, 0, 0, "NORTH"),
    ("2023-03-01", "SKU_SHAMPOO", 50, 18.99, 1, 0, "NORTH"),
    ("2023-04-01", "SKU_SHAMPOO", 55, 18.49, 1, 0, "NORTH"),
    ("2024-01-01", "SKU_SHAMPOO", 60, 20.99, 0, 0, "NORTH"),
    ("2024-02-01", "SKU_SHAMPOO", 65, 19.99, 1, 0, "NORTH"),

    ("2023-01-01", "SKU_FACEWASH", 30, 25.00, 0, 0, "NORTH"),
    ("2023-02-01", "SKU_FACEWASH", 34, 25.00, 0, 0, "NORTH"),
    ("2023-03-01", "SKU_FACEWASH", 38, 23.00, 1, 0, "NORTH"),
    ("2023-04-01", "SKU_FACEWASH", 40, 23.50, 1, 0, "NORTH"),
    ("2024-01-01", "SKU_FACEWASH", 45, 26.00, 0, 0, "NORTH"),
    ("2024-02-01", "SKU_FACEWASH", 50, 24.00, 1, 0, "NORTH"),
]

cur.executemany("""
INSERT INTO sales_history (date, sku, quantity, price, promo, stockout, region)
VALUES (?, ?, ?, ?, ?, ?, ?)
""", sales_data)

inventory_data = [
    ("SKU_SHAMPOO", "Shampoo 100ml", 15, 40),
    ("SKU_FACEWASH", "Facewash 100ml", 20, 30),
]

cur.executemany("""
INSERT INTO inventory (sku, name, stock, threshold)
VALUES (?, ?, ?, ?)
""", inventory_data)

conn.commit()
conn.close()
print("SQLite DB initialized at", DB_PATH)


In [ ]:
# mcp/database/sqlite_handler.py
import sqlite3
import pandas as pd
from pathlib import Path

DB_PATH = Path("mcp/data/sales.db")

def get_connection():
    return sqlite3.connect(DB_PATH)

def load_sales_history_for_skus(skus: list[str]) -> pd.DataFrame:
    conn = get_connection()
    placeholders = ",".join(["?"] * len(skus))
    query = f"""
        SELECT date, sku, quantity, price, promo, stockout, region
        FROM sales_history
        WHERE sku IN ({placeholders})
        ORDER BY date ASC
    """
    df = pd.read_sql_query(query, conn, params=skus)
    conn.close()
    return df

def load_inventory(sku: str | None = None) -> pd.DataFrame:
    conn = get_connection()
    if sku:
        query = "SELECT sku, name, stock, threshold FROM inventory WHERE sku = ?"
        df = pd.read_sql_query(query, conn, params=(sku,))
    else:
        query = "SELECT sku, name, stock, threshold FROM inventory"
        df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def load_inventory_row(sku: str) -> dict | None:
    df = load_inventory(sku)
    if df.empty:
        return None
    row = df.iloc[0]
    return {
        "sku": row["sku"],
        "name": row["name"],
        "stock": int(row["stock"]),
        "threshold": int(row["threshold"]),
    }


In [ ]:
# mcp/llm/azure_llm.py
import os
from langchain_openai import ChatOpenAI

def get_llm(model: str = "azure/genailab-maas-gpt-4o-mini", temp: float = 0.1):
    return ChatOpenAI(
        model=model,
        api_key=os.getenv("AZURE_MAAS_API_KEY"),
        base_url="https://models.inference.ai.azure.com",
        temperature=temp,
    )


In [ ]:
# mcp/vectorstore/chroma_store.py
from pathlib import Path
import chromadb
from chromadb.utils import embedding_functions
from database.sqlite_handler import load_inventory

CHROMA_DIR = Path("mcp/chroma_store")
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

_client = chromadb.PersistentClient(path=str(CHROMA_DIR))

_embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

_collection = _client.get_or_create_collection(
    name="products_inventory",
    embedding_function=_embedding_fn,
)

def sync_inventory_to_chroma(force_rebuild: bool = False):
    global _collection
    if force_rebuild:
        _client.delete_collection("products_inventory")
        _collection = _client.get_or_create_collection(
            name="products_inventory",
            embedding_function=_embedding_fn,
        )

    inv_df = load_inventory()
    if inv_df.empty:
        return

    ids = inv_df["sku"].astype(str).tolist()
    docs = inv_df["name"].astype(str).tolist()
    metadatas = inv_df[["sku", "name"]].to_dict(orient="records")

    _collection.upsert(
        ids=ids,
        documents=docs,
        metadatas=metadatas,
    )

def semantic_search_products(queries: list[str], top_k: int = 3):
    sync_inventory_to_chroma(force_rebuild=False)

    results = []
    for q in queries:
        if not q.strip():
            continue

        res = _collection.query(
            query_texts=[q],
            n_results=top_k,
        )

        ids = res.get("ids", [[]])[0]
        metas = res.get("metadatas", [[]])[0]
        distances = res.get("distances", [[]])[0]

        for sku, meta, dist in zip(ids, metas, distances):
            results.append(
                {
                    "sku": meta.get("sku", sku),
                    "name": meta.get("name", ""),
                    "score": float(dist),
                    "query_phrase": q,
                }
            )

    best_by_sku = {}
    for r in results:
        sku = r["sku"]
        if sku not in best_by_sku or r["score"] < best_by_sku[sku]["score"]:
            best_by_sku[sku] = r

    return list(best_by_sku.values())


In [ ]:
# mcp/agents/guardrails.py
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import re

FORBIDDEN = ["attack", "bomb", "hack", "illegal", "terror"]

def validate_input(text: str) -> None:
    lowered = text.lower()
    for bad in FORBIDDEN:
        if bad in lowered:
            raise Exception("❌ Input contains forbidden/unsafe content.")
    if len(text.strip()) < 5:
        raise Exception("❌ Query too short, please be more specific.")

def extract_horizon_months(text: str) -> int:
    m = re.search(r"(\d+)\s+month", text.lower())
    if m:
        return int(m.group(1))
    return 12

class MonthForecast(BaseModel):
    month: str
    predicted_qty: float
    lower: float | None = None
    upper: float | None = None

class ProductForecast(BaseModel):
    sku: str
    product_name: str
    horizon_months: int
    forecasts: List[MonthForecast]

class ForecastOutput(BaseModel):
    mode: str
    input_used: Dict[str, Any]
    data_preview: Dict[str, Any]
    products: List[ProductForecast]
    llm_raw_text: str

class ReorderDecision(BaseModel):
    reorder_needed: bool
    recommended_quantity: int

class ReorderOutput(BaseModel):
    mode: str
    input_used: Dict[str, Any]
    decision: ReorderDecision
    llm_notes: str

class InventoryOutput(BaseModel):
    mode: str
    input_used: Dict[str, Any]
    status_summary: str

def forecast_output_guardrails(data: dict) -> dict:
    ForecastOutput(**data)
    return data

def reorder_output_guardrails(data: dict) -> dict:
    ReorderOutput(**data)
    return data

def inventory_output_guardrails(data: dict) -> dict:
    InventoryOutput(**data)
    return data


In [ ]:
# mcp/agents/product_resolver.py
import re
from vectorstore.chroma_store import semantic_search_products

def parse_product_names(query: str) -> list[str]:
    lower = query.lower()
    if "for" in lower:
        part = lower.split("for", 1)[1]
    else:
        part = lower

    part = re.split(r"for next|for the next|for upcoming|next \d+ month", part)[0]
    tokens = re.split(r"[\/,]| and ", part)
    names = [t.strip() for t in tokens if t.strip()]
    return names

def resolve_products_to_skus(query: str) -> list[dict]:
    candidate_names = parse_product_names(query)
    if not candidate_names:
        return []

    matches = semantic_search_products(candidate_names, top_k=3)
    simplified = [
        {"sku": m["sku"], "name": m["name"], "score": m["score"]}
        for m in matches
    ]
    simplified.sort(key=lambda x: x["score"])
    return simplified


In [ ]:
# mcp/agents/routing_agent.py
from agents.guardrails import validate_input

def route_query(state: dict) -> dict:
    q = state["query"].lower()
    validate_input(q)

    if "forecast" in q or "predict" in q or "sales for next" in q:
        state["intent"] = "forecast"
    elif "reorder" in q or "restock" in q or "procure" in q:
        state["intent"] = "reorder"
    else:
        state["intent"] = "inventory"

    return state


In [ ]:
# mcp/agents/forecast_agent.py
import pandas as pd
import json
from agents.guardrails import (
    extract_horizon_months,
    forecast_output_guardrails,
)
from agents.product_resolver import resolve_products_to_skus
from database.sqlite_handler import load_sales_history_for_skus
from llm.azure_llm import get_llm

llm = get_llm()

def compress_history_for_prompt(df: pd.DataFrame) -> dict:
    df["date"] = pd.to_datetime(df["date"])
    df["month"] = df["date"].dt.to_period("M")
    grouped = (
        df.groupby(["sku", "month"])
        .agg(
            total_qty=("quantity", "sum"),
            avg_price=("price", "mean"),
            promo_days=("promo", "sum"),
            stockout_days=("stockout", "sum"),
        )
        .reset_index()
    )

    result = {}
    for sku in grouped["sku"].unique():
        sub = grouped[grouped["sku"] == sku].copy()
        result[sku] = sub.to_dict(orient="records")
    return result

def forecasting_agent(state: dict) -> dict:
    query = state["query"]

    horizon_months = extract_horizon_months(query)
    product_matches = resolve_products_to_skus(query)

    if not product_matches:
        raise Exception("Could not resolve any products from your query.")

    skus = [m["sku"] for m in product_matches]
    sku_name_map = {m["sku"]: m["name"] for m in product_matches}

    raw_df = load_sales_history_for_skus(skus)
    if raw_df.empty:
        raise Exception("No historical data found for the selected products.")

    compressed_history = compress_history_for_prompt(raw_df)

    # Build a small data preview for transparency
    data_preview = {}
    for sku in skus:
        sub = raw_df[raw_df["sku"] == sku].head(5).to_dict(orient="records")
        data_preview[sku] = sub

    prompt = f"""
You are an enterprise demand forecasting agent for a CPG distributor.

You receive:
- SKUs and product names
- Historical monthly aggregates (per SKU):
  - month (YYYY-MM)
  - total_qty
  - avg_price
  - promo_days
  - stockout_days

Task:
- Forecast monthly demand for the NEXT {horizon_months} MONTHS for each SKU.
- Consider: trend, seasonality, price, promotions, stockouts.
- Return ONLY strict JSON in this structure:

{{
  "mode": "forecast",
  "input_used": {{
    "horizon_months": {horizon_months},
    "skus": ["{"\", \"".join(skus)}],
    "notes": "LLM-based forecast using compressed history (monthly aggregates)."
  }},
  "data_preview": {{}},
  "products": [
    {{
      "sku": "<sku>",
      "product_name": "<product name>",
      "horizon_months": {horizon_months},
      "forecasts": [
        {{
          "month": "YYYY-MM",
          "predicted_qty": <number>,
          "lower": <number>,
          "upper": <number>
        }}
      ]
    }}
  ],
  "llm_raw_text": ""
}}

Rules:
- Do NOT add explanations outside JSON.
- Months must be continuous from the month after last history.
- All numeric fields must be valid numbers.
- One product block per SKU.
"""

    history_json = json.dumps(compressed_history, default=str)

    messages = [
        {
            "role": "system",
            "content": "You are a precise JSON-only forecasting engine.",
        },
        {
            "role": "user",
            "content": f"SKUs and product names: {json.dumps(sku_name_map)}",
        },
        {
            "role": "user",
            "content": f"Historical monthly aggregates: {history_json}",
        },
        {
            "role": "user",
            "content": prompt,
        },
    ]

    resp = llm.invoke(messages)
    raw_text = resp.content.strip()

    try:
        data = json.loads(raw_text)
    except json.JSONDecodeError:
        cleaned = raw_text.strip("`").replace("json", "")
        data = json.loads(cleaned)

    data["data_preview"] = data_preview
    data["llm_raw_text"] = raw_text

    data = forecast_output_guardrails(data)
    state["output"] = data
    return state


In [ ]:
# mcp/agents/reorder_agent.py
from database.sqlite_handler import load_inventory_row
from llm.azure_llm import get_llm
from agents.guardrails import reorder_output_guardrails

llm = get_llm()

def reorder_agent(state: dict) -> dict:
    query = state["query"]
    sku = state["sku"]

    if not sku:
        raise Exception("For reorder decisions, please provide a SKU explicitly.")

    item = load_inventory_row(sku)
    if item is None:
        state["output"] = {"error": f"SKU {sku} not found in inventory."}
        return state

    stock = item["stock"]
    threshold = item["threshold"]

    reorder_needed = stock < threshold
    if reorder_needed:
        recommended_qty = max(0, threshold * 2 - stock)
    else:
        recommended_qty = 0

    prompt = f"""
You are an enterprise supply chain planner.

Item:
- SKU: {sku}
- Name: {item['name']}
- Current Stock: {stock}
- Reorder Threshold: {threshold}
- Rule-based reorder_needed: {reorder_needed}
- Rule-based recommended_quantity: {recommended_qty}

Explain briefly:
- Why reorder is or is not needed
- Risk of stockout
- Strategic suggestion

Max 120 words.
"""

    resp = llm.invoke(prompt)

    output = {
        "mode": "reorder",
        "input_used": {
            "sku": sku,
            "name": item["name"],
            "stock": stock,
            "threshold": threshold,
        },
        "decision": {
            "reorder_needed": reorder_needed,
            "recommended_quantity": int(recommended_qty),
        },
        "llm_notes": resp.content,
    }

    output = reorder_output_guardrails(output)
    state["output"] = output
    return state


In [ ]:
# mcp/agents/inventory_agent.py
from database.sqlite_handler import load_inventory_row
from llm.azure_llm import get_llm
from agents.guardrails import inventory_output_guardrails

llm = get_llm()

def inventory_agent(state: dict) -> dict:
    sku = state["sku"]
    if not sku:
        raise Exception("Please provide a SKU for inventory health check.")

    item = load_inventory_row(sku)
    if item is None:
        state["output"] = {"error": f"SKU {sku} not found in inventory."}
        return state

    prompt = f"""
You are an inventory health analysis agent.

Item:
- SKU: {sku}
- Name: {item['name']}
- Stock: {item['stock']}
- Threshold: {item['threshold']}

Provide:
- Health status (OK / Low / Critical)
- Risk of stockout
- Next recommended action

Max 80 words.
"""

    resp = llm.invoke(prompt)

    output = {
        "mode": "inventory_health",
        "input_used": {
            "sku": sku,
            "name": item["name"],
            "stock": item["stock"],
            "threshold": item["threshold"],
        },
        "status_summary": resp.content,
    }

    output = inventory_output_guardrails(output)
    state["output"] = output
    return state


In [ ]:
# mcp/graph/workflow.py
from langgraph.graph import StateGraph, END

from agents.routing_agent import route_query
from agents.forecast_agent import forecasting_agent
from agents.reorder_agent import reorder_agent
from agents.inventory_agent import inventory_agent

class AppState(dict):
    pass

graph = StateGraph(AppState)

graph.set_entry_point("route")

graph.add_node("route", route_query)
graph.add_node("forecast", forecasting_agent)
graph.add_node("reorder", reorder_agent)
graph.add_node("inventory", inventory_agent)

graph.add_conditional_edges(
    "route",
    lambda s: s["intent"],
    {
        "forecast": "forecast",
        "reorder": "reorder",
        "inventory": "inventory",
    },
)

graph.add_edge("forecast", END)
graph.add_edge("reorder", END)
graph.add_edge("inventory", END)

app = graph.compile()


In [ ]:
# mcp/run.py
from graph.workflow import app
import json

def main():
    print("=== Enterprise Inventory Intelligence MCP ===")
    query = input("Enter your request:\n> ").strip()
    sku = input("Enter SKU (optional, e.g., SKU_SHAMPOO):\n> ").strip().upper()
    if sku == "":
        sku = None

    state = {
        "query": query,
        "sku": sku,
    }

    result = app.invoke(state)
    output = result.get("output")

    print("\n===== FULL STRUCTURED OUTPUT =====")
    print(json.dumps(output, indent=4, default=str))
    print("==================================\n")

    if isinstance(output, dict) and "mode" in output:
        mode = output["mode"]
        print(f"Mode detected: {mode}")
        if mode == "forecast":
            if output["products"]:
                first = output["products"][0]
                print("\n--- Short forecast preview (first product, first 3 months) ---")
                print(json.dumps(first["forecasts"][:3], indent=4, default=str))
        elif mode == "reorder":
            print("\n--- Reorder decision ---")
            print(json.dumps(output["decision"], indent=4))
        elif mode == "inventory_health":
            print("\n--- Inventory health summary ---")
            print(output["status_summary"])

if __name__ == "__main__":
    main()


pip install -r mcp/requirements.txt
python mcp/init_db.py
python mcp/run.py


This gives you a complete, runnable system with:

Multi-intent LangGraph flow

SQLite backend

Chroma vector search for product matching

LLM-based multi-product forecasting

Reorder logic + LLM reasoning

Inventory health analysis

Strong input/output guardrails

Clear, structured outputs that always show:

What you asked

What data was used

What the system predicted / decided

PROMPT

In [ ]:
You are an **Enterprise Inventory & Forecasting AI Agent** for a CPG (Consumer Packaged Goods) distributor.

Your job is to read **natural language queries** from the user and respond with **STRICT, VALID JSON ONLY**, no extra text.

You support THREE main capabilities:

1. **LLM-based demand forecasting**
2. **Reorder planning**
3. **Inventory health check**

You must always:
- Detect the user’s intent (forecast vs reorder vs inventory health)
- Parse product(s) and time horizon (e.g., “next 12 months”)
- Work for **one or multiple products** at once (e.g., “shampoo/facewash”)
- Apply guardrails on input (disallow unsafe content)
- Apply guardrails on output (strict JSON schema)
- Clearly show:
  - What the user asked
  - Which products/SKUs you operated on
  - Horizon used
  - A small preview of data considered (if available)
  - Final result


====================================================
## 0. INPUT PATTERN (what the user can ask)
====================================================

Examples of user queries you must handle:

- “predict the sales for shampoo/facewash for next 12 months”
- “forecast sales for shampoo, soap and facewash for the next 6 months”
- “do we need to reorder shampoo?”
- “check inventory health for facewash”
- “predict shampoo sales for the upcoming 3 months”
- “reorder decision for SKU_SHAMPOO”
- “check stock status for SKU_FACEWASH”

The user may:
- Refer to products by **name** (“shampoo”, “facewash”)
- Refer to products by **SKU** (“SKU_SHAMPOO”, “SKU_FACEWASH”)
- Ask for **one product** or **several products** in one query
- Specify horizon with phrases like:
  - “next 12 months”
  - “for 6 months”
  - “upcoming 3 months”

If horizon is not found, default to **12 months** for forecasting.


====================================================
## 1. INPUT GUARDRAILS
====================================================

Before doing any reasoning, you must check for unsafe or disallowed content.

Forbidden words (case-insensitive) include (but are not limited to):
- "attack", "bomb", "hack", "illegal", "terror"

If a user query contains such content, you MUST respond with this JSON **exactly**:

{
  "error": "FORBIDDEN_CONTENT",
  "message": "❌ Input contains forbidden or unsafe content."
}

If the user query is too short or unclear (e.g., length < 5 characters), respond with:

{
  "error": "QUERY_TOO_SHORT",
  "message": "❌ Query too short, please be more specific."
}


====================================================
## 2. INTENT DETECTION
====================================================

You must classify each query into exactly one of:

- `"forecast"`
- `"reorder"`
- `"inventory_health"`

Rules (case-insensitive):

- If query contains any of:
  - "forecast", "predict", "sales for next", "predict the sales"
  → intent = "forecast"

- Else if query contains:
  - "reorder", "restock", "procure"
  → intent = "reorder"

- Else:
  → intent = "inventory_health"


====================================================
## 3. TIME HORIZON PARSING (FORECAST ONLY)
====================================================

For forecast intent, you must parse the horizon in **months** from phrases like:

- "next 12 months"
- "for 6 months"
- "for the next 9 months"
- "for upcoming 3 months"

Use this logic:

- If you find a pattern like `(\d+)\s+month` (e.g., 12 months, 6 month, 3 months)
  → horizon_months = that number as integer.
- If no such pattern is found
  → horizon_months = 12.


====================================================
## 4. PRODUCT RESOLUTION (NAMES → SKUs)
====================================================

Users may refer to products with phrases like:
- "shampoo/facewash"
- "shampoo, soap and facewash"
- "shampoo and facewash for next 12 months"

You must:
1. Extract candidate product phrases from the query:
   - Take the part after the first "for" (if present), otherwise use full query.
   - Cut off at horizon phrases like:
     - "for next ... months"
     - "for the next ... months"
     - "for upcoming ..."
     - "next X months"
   - Split remaining text using:
     - '/', ',', ' and '

   Example:
   - "predict the sales for shampoo/facewash for next 12 months"
     → ["shampoo", "facewash"]

   - "forecast sales for shampoo, soap and facewash for the next 6 months"
     → ["shampoo", "soap", "facewash"]

2. For each candidate phrase, semantically map it to one or more SKUs.
   - You SHOULD assume you have access to a **Vector Store (ChromaDB)** built from inventory product names.
   - You must conceptually perform semantic similarity matching between user phrases and product names.
   - For the purpose of output, just assume you successfully resolved a list of SKUs and product names.

3. Final resolved products must be represented as an array of:
   - {"sku": "<SKU_ID>", "name": "<Product Display Name>"}

If you cannot resolve any product:
- Return a JSON error:

{
  "error": "NO_PRODUCTS_RESOLVED",
  "message": "Could not resolve any products from your query. Please mention clear product names."
}


====================================================
## 5. OUTPUT FORMAT (STRICT JSON) – THREE MODES
====================================================

You ***must output only JSON***, no markdown, no explanations outside JSON.

The JSON structure is different for each mode:

-------------------------------------------
### 5.1 FORECAST OUTPUT (mode = "forecast")
-------------------------------------------

When the intent is `"forecast"`, output MUST be:

{
  "mode": "forecast",
  "input_used": {
    "horizon_months": <integer>,
    "skus": ["<SKU1>", "<SKU2>", ...],
    "notes": "LLM-based forecast using compressed historical monthly aggregates."
  },
  "data_preview": {
    "<SKU1>": [
      { "date": "...", "sku": "...", "quantity": <number>, "price": <number>, "promo": <0 or 1>, "stockout": <0 or 1>, "region": "..." },
      ...
    ],
    "<SKU2>": [
      ...
    ]
  },
  "products": [
    {
      "sku": "<SKU1>",
      "product_name": "<Product Name 1>",
      "horizon_months": <integer>,
      "forecasts": [
        {
          "month": "YYYY-MM",
          "predicted_qty": <number>,
          "lower": <number>,
          "upper": <number>
        },
        ...
      ]
    },
    {
      "sku": "<SKU2>",
      "product_name": "<Product Name 2>",
      "horizon_months": <integer>,
      "forecasts": [
        {
          "month": "YYYY-MM",
          "predicted_qty": <number>,
          "lower": <number>,
          "upper": <number>
        },
        ...
      ]
    }
  ],
  "llm_raw_text": "<OPTIONALLY: raw JSON string you internally generated>"
}

Rules for forecast mode:
- `mode` MUST be `"forecast"`.
- `horizon_months` MUST be the parsed horizon (or 12 by default).
- `skus` MUST list all SKUs you forecasted for.
- `data_preview` MUST show at least a few rows per SKU (if historical data is assumed available).
- `products` MUST include one block per SKU.
- In each `forecasts` array:
  - `month` must be in `"YYYY-MM"` format.
  - Months must be sequential from the month immediately after the last historical month, for `horizon_months` steps.
  - All numeric fields must be valid numbers (no NaN, no null, no strings like "N/A").

Guardrail for forecast:
- Before returning, ensure the JSON shape matches the above structure.
- If you detect that structure is invalid, return:

{
  "error": "FORECAST_OUTPUT_SCHEMA_ERROR",
  "message": "Forecast output did not match required JSON schema."
}

-------------------------------------------
### 5.2 REORDER OUTPUT (mode = "reorder")
-------------------------------------------

When the intent is `"reorder"`:

You must:
- Either use the provided SKU
- Or (if not provided) conceptually resolve one product from the query text (similar to product resolution above).

You must output:

{
  "mode": "reorder",
  "input_used": {
    "sku": "<SKU>",
    "name": "<Product Name>",
    "stock": <integer>,
    "threshold": <integer>
  },
  "decision": {
    "reorder_needed": <true or false>,
    "recommended_quantity": <integer>
  },
  "llm_notes": "<short business explanation>"
}

Rules for reorder:
- `reorder_needed` should be `true` if `stock < threshold`, else `false`.
- If `reorder_needed` is true, `recommended_quantity` should be a positive integer (e.g., enough to reach 2x threshold minus current stock).
- `llm_notes` should briefly explain:
  - Why reorder is or isn’t needed
  - Risk of stockout
  - Strategic suggestion (e.g., consider seasonality/promotions)

Guardrail for reorder:
- If the final JSON does not match exactly:
  - `mode`, `input_used`, `decision`, `llm_notes` shape, respond with:

{
  "error": "REORDER_OUTPUT_SCHEMA_ERROR",
  "message": "Reorder output did not match required JSON schema."
}

-------------------------------------------
### 5.3 INVENTORY HEALTH OUTPUT (mode = "inventory_health")
-------------------------------------------

When the intent is `"inventory_health"`:

You must output:

{
  "mode": "inventory_health",
  "input_used": {
    "sku": "<SKU>",
    "name": "<Product Name>",
    "stock": <integer>,
    "threshold": <integer>
  },
  "status_summary": "<short health summary>"
}

- `status_summary` should:
  - Mention whether stock is OK / Low / Critical.
  - Mention risk of stockout.
  - Suggest a next action (e.g., monitor, reorder soon, no action).

Guardrail:
- If the structure doesn’t match exactly, return:

{
  "error": "INVENTORY_OUTPUT_SCHEMA_ERROR",
  "message": "Inventory health output did not match required JSON schema."
}


====================================================
## 6. FORECASTING LOGIC (LLM-BASED)
====================================================

When intent = `"forecast"`:

1. Parse `horizon_months` from user query.
2. Resolve product phrases → SKUs & names (via conceptual vector search).
3. Assume you have access to **historical sales data** per SKU, with fields:
   - date
   - sku
   - quantity
   - price
   - promo
   - stockout
   - region

4. Conceptually compress raw history into **monthly aggregates** for each SKU:
   - month (YYYY-MM)
   - total_qty
   - avg_price
   - promo_days
   - stockout_days

5. Use your internal LLM reasoning to:
   - Analyze trend, seasonality, price sensitivity, promo impact, and stockout days.
   - Produce a monthly forecast for the requested horizon.

6. Fill the JSON as described in section 5.1.

Even if you cannot literally query a real database, you must still:
- Act as if historical data exists.
- Show a plausible `data_preview` and `forecasts` consistent with the query and descriptions.


====================================================
## 7. BEHAVIOR SUMMARY
====================================================

- ALWAYS respond in **JSON only**, no plain text explanation outside JSON.
- ALWAYS detect intent: forecast / reorder / inventory_health.
- ALWAYS enforce input guardrails and return error JSON if invalid.
- For forecasting:
  - Support multiple products at once.
  - Use horizon in months.
  - Return a list of product forecasts with one block per SKU.
- For reorder:
  - Decide `reorder_needed` using stock vs threshold.
  - Provide recommended quantity and business notes.
- For inventory health:
  - Provide concise health summary and next actions.
- If anything breaks schema or cannot be understood, return a JSON error with:
  - `"error": "<SOME_ERROR_CODE>"`
  - `"message": "<human-readable explanation>"`.

You must follow this behavior consistently for ALL user queries.
